In [ ]:
import pandas as pd
df = pd.read_csv('./email.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['Category'].unique()

In [ ]:
df['Category'].value_counts()

In [ ]:
diff_rows = df[df['Category'].isin(['ham', 'spam'])]
df = diff_rows

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Category'] = encoder.fit_transform(df['Category'])
df = df.rename(columns={'Category': 'spam', 'Message': 'message'})
df.head()

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates(keep='first')
df.shape

**Analysis**

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.bar(['Ham', 'Spam'], df['spam'].value_counts())
plt.subplot(1, 2, 2)
plt.pie(df['spam'].value_counts(), labels=['Ham', 'Spam'], autopct='%1.1f%%')
plt.show()

In [ ]:
df["chars"] = df.apply(lambda x: len(x["message"]), axis=1)
df.head()

In [ ]:
df[df.spam == 0].describe()

In [ ]:
df[df.spam == 1].describe()

**preprocessing**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

stemmer = PorterStemmer()

df = df.reset_index(drop=True)

messages = df['message']
spam = df['spam']

corpus = []
for i in range(len(messages)):
    data = re.sub('[^a-zA-Z0-9]',' ',messages[i])
    data = data.lower()
    data = data.split()
    data = [stemmer.stem(word)for word in data if word not in set(stopwords.words('english'))]
    data = ' '.join(data)
    corpus.append(data)
df["new-msg"] = corpus
df.head()

**model**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer()
# tf = TfidfVectorizer()
x = cv.fit_transform(corpus).toarray()
x.shape

In [ ]:
y = df.spam.values

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix

bb = BernoulliNB()
nb = MultinomialNB()
rf = RandomForestClassifier()

In [ ]:
bb.fit(x_train,y_train)
y_pred1 = bb.predict(x_test)
print(accuracy_score(y_pred1,y_test))
print(confusion_matrix(y_pred1,y_test))
print(precision_score(y_pred1,y_test))

In [ ]:
nb.fit(x_train,y_train)
y_pred2 = nb.predict(x_test)
print(accuracy_score(y_pred2,y_test))
print(confusion_matrix(y_pred2,y_test))
print(precision_score(y_pred2,y_test))

In [ ]:
rf.fit(x_train,y_train)
y_pred3 = rf.predict(x_test)
print(accuracy_score(y_pred3,y_test))
print(confusion_matrix(y_pred3,y_test))
print(precision_score(y_pred3,y_test))

**save model**

In [ ]:
import pickle
with open('../model.pkl', 'wb') as f:
    pickle.dump(nb, f)

with open('../vectorizer.pkl', 'wb') as f:
    pickle.dump(cv, f)